In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
import json
import requests

In [2]:
# Mapbox API token
token = 'pk.eyJ1Ijoiemhlbmh1YSIsImEiOiJjazJjZnU2d2UwZHp6M2RvMmhhOGN6cG43In0.9F-J0PB0VUlBLxLE-TE_Tw'


In [3]:
# Extract routing info given coordinates using Mapbox API

def obtainRouting(starting_point, end_point):
    coordinates = str(starting_point[1]) + '%2C' + str(starting_point[0]) + '%3B' + str(end_point[1]) + '%2C' + str(end_point[0])

    # Obtain routing info
    link = 'https://api.mapbox.com/directions/v5/mapbox/driving/'
    response = requests.get(link+coordinates+
                            '?alternatives=true&geometries=geojson&steps=true&'+
                            'access_token='+token)
    data = response.json()

    # Change routing list to dataframe
    # Input: [[-73.99604, 40.730953], [-73.998611, 40.732216], [-73.998611, 40.732216], ...]
    # Output: dataframe
    routing = data['routes'][0]['geometry']['coordinates']
    coord = pd.DataFrame()
    lat_list = []
    lon_list = []
    for i in range(len(routing)):
        lat_list.append(routing[i][1])
        lon_list.append(routing[i][0])
    coord['lat'] = lat_list
    coord['lon'] = lon_list
    
    return coord

def plotRouting(coord, color, name, legendgroup, showlegend):
    trace = go.Scattermapbox(lat=coord['lat'], 
                             lon=coord['lon'],
                             mode='lines',
                             marker=go.scattermapbox.Marker(size=8,
                                                            color=color,
                                                            opacity=0.5
                                                            ),
                             hoverinfo='skip',
                             name=name,
                             legendgroup=legendgroup,
                             showlegend=showlegend,
                            )
    return trace

def addStop(stop, color, opacity, text, name, legendgroup, showlegend):
    stop = go.Scattermapbox(lat=[stop[0]],
                            lon=[stop[1]],
                            mode='markers',
                            marker=go.scattermapbox.Marker(size=16,
                                                           color=color,
                                                           opacity=opacity,
                                                          ),
                            hoverinfo='all',
                            name=name,
                            legendgroup=legendgroup,
                            showlegend=showlegend,
                            hovertemplate =
                            '<b>%{lat:.2f}, %{lon:.2f}</b><br><br>' +
                            '<b>%{text}</b>',
                            text = [text],
                           )
    return stop

In [21]:
# Original map, update legend
fig = go.Figure()

fig.update_layout(hovermode='closest',
                  mapbox_style='light', 
                  mapbox_accesstoken=token,
                  mapbox_zoom=3.8, 
                  mapbox=dict(center=go.layout.mapbox.Center(lat=41, lon=-115.199310),
                              bearing=0))

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
                  legend_orientation="v",
                  legend=dict(
                      x=-0.25,
                      y=0.5,
                      traceorder="normal",
                      font=dict(
                          size=16,
                          color="#003152"
                      ),
                      borderwidth=0
                  )
                 )

# June 15
bay = [37.427427, -122.199310]
fig.add_trace(addStop(bay, color="#003152", 
                      opacity=0.7,
                      name='Stops',
                      legendgroup='Stops',
                      showlegend=True,
                      text='June 15'))
fig.add_trace(addStop(bay, color="#003152", 
                      opacity=0,
                      name='Bay Area',
                      legendgroup='Stops',
                      showlegend=False,
                      text='June 15'))

# June 16
davis = [38.546340, -121.726533]
fig.add_trace(plotRouting(obtainRouting(bay, [45.397647, -122.270569]), 
                          color='#D5B5A4', 
                          name='Aborted',
                          legendgroup="Aborted",
                          showlegend=True))
fig.add_trace(plotRouting(obtainRouting(bay, [32.759912, -117.206380]), 
                          color='#D5B5A4', 
                          name='Aborted', 
                          legendgroup="Aborted",
                          showlegend=False))
fig.add_trace(plotRouting(obtainRouting(bay, davis), 
                          color='#db4d6d', 
                          name='Adopted',
                          legendgroup='Routes',
                          showlegend=True))
fig.add_trace(addStop(davis, color="#003152",
                      opacity=0.7,
                      name='Davis, CA',
                      legendgroup='Stops',
                      showlegend=False,
                      text='June 16'))


# June 17-26
utah = [40.679018, -112.26300]
fig.add_trace(plotRouting(obtainRouting(utah, [39.332114, -106.987696]), 
                          color='#D5B5A4', 
                          name='Aborted', 
                          legendgroup='Aborted',
                          showlegend=False))
fig.add_trace(plotRouting(obtainRouting(utah, [37.073156, -113.583815]), 
                          color='#D5B5A4', 
                          name='Aborted', 
                          legendgroup='Aborted',
                          showlegend=False))
fig.add_trace(plotRouting(obtainRouting(utah, [42.563728, -114.479309]), 
                          color='#D5B5A4', 
                          name='Aborted', 
                          legendgroup='Aborted',
                          showlegend=False))
fig.add_trace(plotRouting(obtainRouting(davis, utah), 
                          color='#db4d6d', 
                          name='June 16', 
                          legendgroup='Routes',
                          showlegend=False))
fig.add_trace(addStop(utah, color="#003152", 
                      opacity=0.7,
                      name='Lake Point, UT',
                      legendgroup='Stops',
                      showlegend=False,
                      text='June 17-26'))

# June 27-28
parkCity = [40.699425, -111.566409]
fig.add_trace(plotRouting(obtainRouting(utah, parkCity), 
                          color='#003152', 
                          name='With Martin', 
                          legendgroup='w/Martin',
                          showlegend=True))
fig.add_trace(addStop(parkCity, color="#003152", 
                      opacity=0.7,
                      name='Park City, UT',
                      legendgroup='Stops',
                      showlegend=False,
                      text='June 27-28'))

# June 29-30
yellowstone = [44.532704, -110.435011]
fig.add_trace(plotRouting(obtainRouting(parkCity, yellowstone), 
                          color='#003152', 
                          name='June 29', 
                          legendgroup='w/Martin',
                          showlegend=False))
fig.add_trace(addStop(yellowstone, color="#003152", 
                      opacity=0.7,
                      name='Yellowstone National Park, WY',
                      legendgroup='Stops',
                      showlegend=False,
                      text='June 29-30'))

# July 1
grandteton = [43.753074, -110.719810]
slc = [40.776485, -111.981147]
fig.add_trace(plotRouting(obtainRouting(yellowstone, grandteton), 
                          color='#D5B5A4', 
                          name='Aborted', 
                          legendgroup='Aborted',
                          showlegend=False))
fig.add_trace(plotRouting(obtainRouting(grandteton, slc), 
                          color='#D5B5A4', 
                          name='Aborted', 
                          legendgroup='Aborted',
                          showlegend=False))
glacier = [48.385916, -114.072221]
glacier_2 = [48.768817, -114.283210]
fig.add_trace(plotRouting(obtainRouting(yellowstone, glacier), 
                          color='#003152', 
                          name='July 1', 
                          legendgroup='w/Martin',
                          showlegend=False))
fig.add_trace(plotRouting(obtainRouting(glacier, glacier_2), 
                          color='#003152', 
                          name='July 1', 
                          legendgroup='w/Martin',
                          showlegend=False))
fig.add_trace(addStop(glacier, color="#003152", 
                      opacity=0.7,
                      name='Glacier National Park, MT',
                      legendgroup='Stops',
                      showlegend=False,
                      text='July 1'))

# July 2
greatfalls = [47.501290, -111.299398]
fig.add_trace(plotRouting(obtainRouting(glacier, greatfalls), 
                          color='#D5B5A4', 
                          name='Aborted', 
                          legendgroup='Aborted',
                          showlegend=False))
kellog = [47.540682, -116.124862]
fig.add_trace(plotRouting(obtainRouting(glacier, kellog), 
                          color='#003152', 
                          name='July 2', 
                          legendgroup='w/Martin',
                          showlegend=False))
fig.add_trace(addStop(kellog, color="#003152", 
                      opacity=0.7,
                      name='Kellog, ID',
                      legendgroup='Stops',
                      showlegend=False,
                      text='July 2'))

In [22]:
# July 3-5
missoula = [46.864275, -113.996716]
idahofalls = [43.496427, -112.053929]
fig.add_trace(plotRouting(obtainRouting(idahofalls, missoula), 
                          color='#D5B5A4', 
                          name='Aborted', 
                          legendgroup='Aborted',
                          showlegend=False))

seattle_1 = [47.665097, -122.313306]
seattle_2 = [47.495807, -122.313594]
glass = [48.142749, -122.785049]
fig.add_trace(plotRouting(obtainRouting(kellog, seattle_1), 
                          color='#003152', 
                          name='July 3', 
                          legendgroup='w/Martin',
                          showlegend=False))
fig.add_trace(plotRouting(obtainRouting(seattle_1, seattle_2), 
                          color='#003152', 
                          name='July 4', 
                          legendgroup='w/Martin',
                          showlegend=False))
fig.add_trace(plotRouting(obtainRouting(seattle_2, glass), 
                          color='#003152', 
                          name='July 5', 
                          legendgroup='w/Martin',
                          showlegend=False))
fig.add_trace(addStop(seattle_1, color="#003152", 
                      opacity=0.7,
                      name='Seattle, WA',
                      legendgroup='Stops',
                      showlegend=False,
                      text='July 3'))
fig.add_trace(addStop(seattle_2, color="#003152", 
                      opacity=0.7,
                      name='Sea-Tac, WA',
                      legendgroup='Stops',
                      showlegend=False,
                      text='July 4-5'))

# July 6-12
albany = [44.616812, -123.097559]
fig.add_trace(plotRouting(obtainRouting(seattle_2, albany), 
                          color='#db4d6d', 
                          name='June 6', 
                          legendgroup='Routes',
                          showlegend=False))
fig.add_trace(addStop(albany, color="#003152", 
                      opacity=0.7,
                      name='Albany, OR',
                      legendgroup='Stops',
                      showlegend=False,
                      text='July 6-12'))


In [23]:
# July 13
crater = [42.871264, -122.169099]
fig.add_trace(plotRouting(obtainRouting(albany, crater), 
                          color='#db4d6d', 
                          name='June 13', 
                          legendgroup='Routes',
                          showlegend=False))
fig.add_trace(addStop(crater, color="#003152", 
                      opacity=0.7,
                      name='Crater Lake National Park, OR',
                      legendgroup='Stops',
                      showlegend=False,
                      text='July 13'))


In [24]:
# Push to Plotly
import chart_studio

username = 'jamesjplus' # your username
api_key = '5mb9BzjI0ffrT6kaCotT' # your api key - go to profile > settings > regenerate key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

import chart_studio.plotly as py
py.plot(fig, filename = '2020summer', auto_open=False)

'https://plotly.com/~jamesjplus/1/'